# Mutual Information (MNIST)

Эксперименты с оценкой энтропии для данных рукописных цифр.

# Преамбула

### Tensorflow

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

tfds.disable_progress_bar()
tf.enable_v2_behavior()

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

### Math, Numpy, Scipy, Pandas

In [ ]:
import math
import numpy as np
import scipy as sp
import scipy.stats as sps
import scipy.linalg as spl
import pandas as pd

### Matplotlib, Seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Sklearn

In [ ]:
# Метод главных компонент.
from sklearn.decomposition import PCA

# Выбор модели по кросс-валидации (поиск по сетке).
from sklearn.model_selection import GridSearchCV

### Joblib

In [ ]:
from joblib import Parallel, delayed

global_n_jobs = 16

### OS, shutil, Json, CSV, copy

In [ ]:
import os
import shutil
import json
import csv
import copy

### Mutinfo

In [ ]:
import sys
sys.path.insert(0, './py')

In [ ]:
from mutinfo.estimators.mutual_information import MutualInfoEstimator
from mutinfo.keras.layers import TunableGaussianNoise

## Вспомогательное

In [ ]:
# Глобальная информация.
global_info = dict()

In [ ]:
# Информация об опыте.
info = dict()

In [ ]:
def normalize_uint8(data, label):
    """Нормализация: `uint8` -> `float32`."""
    return tf.cast(data, tf.float32) / 255.0, label

In [ ]:
def imshow_array(array):
    """Отображение массива нормированных пикселей."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

In [ ]:
def dataset_Y_to_X(X, Y):
    """Поменять у датасета пары (X, Y) на (X, X) (нужно, например, для обучения автоэнкодера)."""
    return X, X

In [ ]:
def concave_loss(y_true, y_pred):
    """Вогнутая функция потерь, дающая более четкие изображения при обучении."""
    delta = tf.keras.backend.abs(y_true - y_pred)
    squared = tf.keras.backend.square(y_true - y_pred)
    return tf.keras.backend.mean(delta - 0.5 * squared, axis=-1)

## Путь к папке с данными

In [ ]:
#path = "/content/drive/My Drive/Information_v2/"
path = os.path.abspath(os.getcwd()) + "/data/"

In [ ]:
experiments_path = path + "mutual_information/MNIST/"
models_path = experiments_path + "models/"

In [ ]:
mnist_shape = (28, 28, 1)

### Полный набор данных

In [ ]:
(ds_full_train, ds_full_test), ds_info = tfds.load(
    'mnist',
    data_dir=path + 'tensorflow_datasets/',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
ds_full_train = ds_full_train.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_full_test  = ds_full_test.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
ds_train = ds_full_train.take(60000)
ds_train = np.array([sample for sample in ds_train])

ds_test  = ds_full_test.take(60000)
ds_test  = np.array([sample for sample in ds_test])

In [ ]:
ds_train_X = ds_train[:,0]
ds_test_X  = ds_test[:,0]

In [ ]:
ds_train_Y = ds_train[:,1]
ds_test_Y = ds_test[:,1]

## Автокодировщик для изображений

### Тренировочные и тестовые наборы

In [ ]:
ae_batch_size = 2048

In [ ]:
(ds_ae_train, ds_ae_test), ds_info = tfds.load(
    'mnist',
    data_dir=path + 'tensorflow_datasets/',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
ds_ae_train = ds_ae_train.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_ae_train = ds_ae_train.map(dataset_Y_to_X, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_ae_train = ds_ae_train.cache()
ds_ae_train = ds_ae_train.shuffle(ds_info.splits['train'].num_examples)
ds_ae_train = ds_ae_train.batch(ae_batch_size)
ds_ae_train = ds_ae_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_ae_test = ds_ae_test.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_ae_test = ds_ae_test.map(dataset_Y_to_X, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_ae_test = ds_ae_test.batch(ae_batch_size)
ds_ae_test = ds_ae_test.cache()
ds_ae_test = ds_ae_test.prefetch(tf.data.experimental.AUTOTUNE)

### Автокодировщик

In [ ]:
# РАЗМЕРНОСТЬ КОДА.
# #
# #

codes_dim_X = 10 # MNSIT

# #
# #

In [ ]:
def cnn_autoencoder(shape_input, dimension):
    # Инициализация весов.
    init = tf.keras.initializers.RandomNormal(stddev = 1.0)

    # Входные данные генератора / выборки.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer

    # 1 блок слоёв.
    next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 12, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)

    # 2 блок слоёв.
    #next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 18, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)

    # 3 блок слоёв.
    next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 27, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)

    # Бутылочное горлышко.
    next_layer = tf.keras.layers.Flatten()(next_layer)
    next_layer = tf.keras.layers.Dense(dimension)(next_layer)
    bottleneck = tf.keras.layers.Activation('tanh')(next_layer)

    # Модель кодировщика.
    encoder = tf.keras.Model(input_layer, bottleneck)

    # Начало модели декодировщика.
    input_code_layer = tf.keras.layers.Input((dimension))
    next_layer = input_code_layer

    # 3 блок слоёв.
    #next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.Dense(4*4*27)(next_layer)
    next_layer = tf.keras.layers.Reshape((4, 4, 27))(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.2)(next_layer)

    # 2 блок слоёв.
    #next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 18, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.Cropping2D(cropping=((0, 1), (0, 1)))(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.1)(next_layer)

    # 1 блок слоёв.
    next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 12, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1)(next_layer)

    # 0 блок слоёв.
    #next_layer = tf.keras.layers.GaussianNoise(0.1)(next_layer)
    next_layer = tf.keras.layers.UpSampling2D(size=(2, 2))(next_layer)
    next_layer = tf.keras.layers.Conv2D(filters = 1, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.Activation('sigmoid')(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.1)(next_layer)

    output_layer = next_layer

    # Модель.
    decoder = tf.keras.models.Model(input_code_layer, output_layer) # Декодировщик.
    autoencoder = tf.keras.Sequential([encoder, decoder])

    # Компиляция модели.
    opt = tf.keras.optimizers.Adam(learning_rate = 5e-3)
    autoencoder.compile(loss = concave_loss, optimizer = opt, loss_weights = [1.0])
    return encoder, decoder, autoencoder

In [ ]:
load_X_autoencoder = True#False

In [ ]:
if load_X_autoencoder:
    encoder_X = tf.keras.models.load_model(models_path + "autoencoder/encoder_X.h5")
    decoder_X = tf.keras.models.load_model(models_path + "autoencoder/decoder_X.h5")
    autoencoder_X = tf.keras.Sequential([encoder_X, decoder_X])
    autoencoder_X.compile(loss = concave_loss, optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

In [ ]:
if not load_X_autoencoder:
    encoder_X, decoder_X, autoencoder_X = cnn_autoencoder(mnist_shape, codes_dim_X)
    
    autoencoder_X.fit(
        ds_ae_train,
        epochs=300,
        validation_data=ds_ae_test
    )
    
    # Сохранение моделей.
    autoencoder_X.save(models_path + "/autoencoder/autoencoder_X.h5")
    encoder_X.save(models_path + "/autoencoder/encoder_X.h5")
    decoder_X.save(models_path + "/autoencoder/decoder_X.h5")

## Классификатор изображений

In [ ]:
epochs_counter = 0

In [ ]:
# Номер исследуемого слоя.
layer_index = 5

# Стандартное отклонение шума, добавляемого к слоям.
layers_noise_std = 5e-2

In [ ]:
next_epoch = 200

In [ ]:
delta_epochs = next_epoch - epochs_counter
epochs_counter = next_epoch

In [ ]:
dataset_path = experiments_path + ("%.1e" % layers_noise_std) + "/" + "layer_" + str(layer_index) + "/" + str(epochs_counter) + "/"

In [ ]:
full_path = dataset_path + "autoencoders/"
os.makedirs(full_path, exist_ok=True)

### Тренировочные и тестовые наборы

In [ ]:
cl_batch_size = 5000

In [ ]:
(ds_cl_train, ds_cl_test), ds_info = tfds.load(
    'mnist',
    data_dir=path + 'tensorflow_datasets/',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
ds_cl_train = ds_cl_train.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_cl_train = ds_cl_train.cache()
ds_cl_train = ds_cl_train.shuffle(ds_info.splits['train'].num_examples)
ds_cl_train = ds_cl_train.batch(cl_batch_size)
ds_cl_train = ds_cl_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_cl_test = ds_cl_test.map(normalize_uint8, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_cl_test = ds_cl_test.batch(cl_batch_size)
ds_cl_test = ds_cl_test.cache()
ds_cl_test = ds_cl_test.prefetch(tf.data.experimental.AUTOTUNE)

### Классификатор

In [ ]:
def convolutional_classifier(shape_input):
    # Инициализация весов.
    init = tf.keras.initializers.RandomNormal(stddev = 0.02)

    # Входные данные генератора / выборки.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_0')(next_layer)

    # 1 блок слоёв.  
    next_layer = tfa.layers.SpectralNormalization(
        tf.keras.layers.Conv2D(
            filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init
        ),
        name='SN_1'
    )(next_layer)
    
    #next_layer = tf.keras.layers.BatchNormalization()(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1, name='Dropout_1')(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_1')(next_layer)

    output_layer_1 = next_layer

    # 2 блок слоёв. 
    next_layer = tfa.layers.SpectralNormalization(
        tf.keras.layers.Conv2D(
            filters = 32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init
        ),
        name='SN_2'
    )(next_layer)
    
    next_layer = tf.keras.layers.BatchNormalization(name='BatchNormalization_2')(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1, name='Dropout_2')(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_2')(next_layer)

    output_layer_2 = next_layer

    # 3 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(
        tf.keras.layers.Conv2D(
            filters = 16, kernel_size = (3, 3), strides = (1, 1), padding = 'same', kernel_initializer = init
        ),
        name='SN_3'
    )(next_layer)
    
    next_layer = tf.keras.layers.BatchNormalization(name='BatchNormalization_3')(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1, name='Dropout_3')(next_layer)
    next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(next_layer)
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_3')(next_layer)

    output_layer_3 = next_layer
    
    # 4 блок слоёв.
    next_layer = tf.keras.layers.Flatten()(next_layer)
    
    next_layer = tfa.layers.SpectralNormalization(
        tf.keras.layers.Dense(16),
        name='SN_4'
    )(next_layer)
    
    next_layer = tf.keras.layers.BatchNormalization(name='BatchNormalization_4')(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_4')(next_layer)

    output_layer_4 = next_layer
    
    # 5 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(
        tf.keras.layers.Dense(10),
        name='SN_5'
    )(next_layer)
    
    next_layer = tf.keras.layers.BatchNormalization(name='BatchNormalization_5')(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Activation('tanh')(next_layer)
    next_layer = TunableGaussianNoise(layers_noise_std, name='GaussianNoise_5')(next_layer)

    output_layer_5 = next_layer

    # Вывод.
    #next_layer = tf.keras.layers.Flatten()(next_layer)
    next_layer = tf.keras.layers.Dense(10)(next_layer)
    output_layer = tf.keras.layers.Activation('softmax')(next_layer)

    # Модель.
    debug_model = tf.keras.models.Model([input_layer],
                                        [output_layer_1,
                                         output_layer_2,
                                         output_layer_3,
                                         output_layer_4,
                                         output_layer_5])
    model = tf.keras.models.Model(input_layer, output_layer)

    # Компиляция модели.
    opt = tf.keras.optimizers.Adam(lr = 1e-3)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = opt,
                  loss_weights = [1.0], metrics=['accuracy'])
    return model, debug_model

In [ ]:
load_X_classifier = False

In [ ]:
# Загрузка модели.
if load_X_classifier:
    classifier = tf.keras.models.load_model(models_path + "/classifier/classifier.h5")
    debug_classifier = tf.keras.models.load_model(models_path + "/classifier/debug_classifier.h5")

In [ ]:
if not load_X_classifier:
    classifier, debug_classifier = convolutional_classifier(mnist_shape)
    # Сводка по модели.
    classifier.summary()
    # Отрисовка модели.
    #tf.keras.utils.plot_model(classifier, show_shapes = True, show_layernames = True)

In [ ]:
history_callback = classifier.fit(
    ds_cl_train,
    epochs=delta_epochs,
    validation_data=ds_cl_test
)

In [ ]:
loss_history = np.array(history_callback.history["loss"])
val_loss_history = np.array(history_callback.history["val_loss"])
accuracy_history = np.array(history_callback.history["accuracy"])
val_accuracy_history = np.array(history_callback.history["val_accuracy"])

info['last_loss'] = loss_history[-1]
info['last_val_loss'] = val_loss_history[-1]
info['last_accuracy'] = accuracy_history[-1]
info['last_val_accuracy'] = val_accuracy_history[-1]

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

# Оценка взаимной информации

### Получение значений слоя

In [ ]:
# Приходится делать predict по частям.
min_batch_number = 10

_splitted = tf.split(tf.stack(ds_train_X), min_batch_number)
_layer_predicted_train = tf.concat([debug_classifier(_splitted[i])[layer_index - 1] for i in range(min_batch_number)], 0)

_splitted = tf.split(tf.stack(ds_test_X), min_batch_number)
_layer_predicted_test = tf.concat([debug_classifier(_splitted[i])[layer_index - 1] for i in range(min_batch_number)], 0)

In [ ]:
ds_train_L = np.array([_layer_predicted_train[i].numpy().flatten() for i in range(_layer_predicted_train.shape[0])])
ds_test_L  = np.array([_layer_predicted_test[i].numpy().flatten() for i in range(_layer_predicted_test.shape[0])])

## Автокодировщик

Сжатие данных предлагается делать автокодировщиком.
Для архитектуры специфицируется только формат входных данных, а также размерность внутреннего представления (кодов).

In [ ]:
# РАЗМЕРНОСТЬ КОДА.
# #
# #

codes_dim_L = 4  # Слой.

# #
# #

In [ ]:
# Число эпох для обучения.
autoencoders_epochs = 2000

In [ ]:
info['autoencoders_epochs'] = autoencoders_epochs

### Автокодировщик для слоя

In [ ]:
def dense_autoencoder(shape_input, dimension):
    # Инициализация весов.
    init = tf.keras.initializers.RandomNormal(stddev = 0.02)

    # Входные данные генератора / выборки.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer
    next_layer = tf.keras.layers.GaussianNoise(0.02)(next_layer)

    # 1 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(512, kernel_initializer = init),
                                                  power_iterations = 3)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1)(next_layer)

    # 2 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(256, kernel_initializer = init),
                                                  power_iterations = 3)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    
    # 3 блок слоёв.
    #next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(128, kernel_initializer = init),
    #                                              power_iterations = 3)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    
    # 4 блок слоёв.
    #next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(32, kernel_initializer = init),
    #                                              power_iterations = 3)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    #next_layer = tf.keras.layers.Dropout(0.1)(next_layer)
    
    # Бутылочное горлышко.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(dimension),
                                                  power_iterations = 3)(next_layer)
    bottleneck = tf.keras.layers.Activation('tanh', name='bottleneck')(next_layer)

    # Модель кодировщика.
    encoder = tf.keras.Model(input_layer, bottleneck)

    # Начало модели декодировщика.
    input_code_L = tf.keras.layers.Input((dimension))
    next_layer = input_code_L
    
    # 4 блок слоёв.
    #next_layer = tf.keras.layers.Dense(32, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 3 блок слоёв.
    #next_layer = tf.keras.layers.Dense(128, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 2 блок слоёв.
    next_layer = tf.keras.layers.Dense(256, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 1 блок слоёв.
    next_layer = tf.keras.layers.Dense(512, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 0 блок слоёв.
    next_layer = tf.keras.layers.Dense(shape_input[0])(next_layer) # Подразумевается, что вход - всё равно вектор.
    #next_layer = tf.keras.layers.Activation('tanh')(next_layer)
    
    output_layer = next_layer
    
    # Модель.
    decoder = tf.keras.models.Model(input_code_L, output_layer) # Декодировщик.
    autoencoder = tf.keras.Sequential([encoder, decoder])

    # Компиляция модели.
    opt = tf.keras.optimizers.Adam(learning_rate = 2e-3)
    autoencoder.compile(loss = 'mse', optimizer = opt)
    
    return encoder, decoder, autoencoder

In [ ]:
load_L_autoencoder = False

In [ ]:
if load_L_autoencoder:
    encoder_L = tf.keras.models.load_model(full_path + "encoder_L.h5")
    decoder_L = tf.keras.models.load_model(full_path + "decoder_L.h5")
    autoencoder_L = tf.keras.Sequential([encoder_L, decoder_L])
    autoencoder.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

    with open(full_path + 'info.json', 'r') as fp:
        info = json.load(fp)

In [ ]:
if not load_L_autoencoder:
    encoder_L, decoder_L, autoencoder_L = dense_autoencoder((ds_train_L.shape[1],), codes_dim_L)
    
    autoencoder_L.fit(
    ds_train_L,
    ds_train_L,
    epochs=autoencoders_epochs,
    validation_data=(ds_test_L, ds_test_L),
    batch_size=ds_train_L.shape[0] // 10)
    
    # Сохранение моделей.
    autoencoder_L.save(full_path + "autoencoder_L.h5")
    encoder_L.save(full_path + "encoder_L.h5")
    decoder_L.save(full_path + "decoder_L.h5")
    
    # Сохранение информации.
    with open(full_path + 'info.json', 'w') as fp:
        json.dump(info, fp, indent=4)

### Получение кодов всех элементов набора данных

In [ ]:
# Вход классификатора
_splitted = tf.split(tf.stack(ds_train_X), min_batch_number)
codes_X = tf.concat([encoder_X(_splitted[i]) for i in range(min_batch_number)], 0)

In [ ]:
# Выход слоя
codes_L = np.array(encoder_L.predict(ds_train_L))

In [ ]:
# Совместный датасет для входа классификатора и выхода слоя
codes_X_L = np.concatenate((codes_X, codes_L), 1)
codes_dim_X_L = codes_dim_L + codes_dim_X

In [ ]:
PCA_codes_X = PCA(n_components=codes_dim_X, whiten=True)
codes_pca_X = np.array(PCA_codes_X.fit_transform(codes_X))

PCA_codes_L = PCA(n_components=codes_dim_L, whiten=True)
codes_pca_L = np.array(PCA_codes_L.fit_transform(codes_L))

In [ ]:
codes_pca_X_L = np.concatenate((codes_pca_X, codes_pca_L), 1)

In [ ]:
pp = sns.pairplot(pd.DataFrame(codes_pca_X_L[0:10000]), height = 2.0, aspect=1.6,
                      plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.05, size=0.01, s=0.01),
                      diag_kind="kde", diag_kws=dict(shade=True))

fig = pp.fig
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Pairwise Plots', fontsize=14)

## Подсчёт взаимной информации

In [ ]:
X_L_mi_estimator = MutualInfoEstimator(n_jobs = global_n_jobs)

In [ ]:
X_L_mi_estimator.fit(codes_pca_X, codes_pca_L, verbose=10)

In [ ]:
mutual_information_X_L, mutual_information_X_L_error = X_L_mi_estimator.predict(codes_pca_X, codes_pca_L,
                                                                                verbose=10)

In [ ]:
info['mutual_information_X_L'] = mutual_information_X_L
info['mutual_information_X_L_error'] = mutual_information_X_L_error

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
L_Y_mi_estimator = MutualInfoEstimator(n_jobs = global_n_jobs, Y_is_discrette=True)

In [ ]:
L_Y_mi_estimator.fit(codes_pca_L, ds_train_Y, verbose=10)

In [ ]:
mutual_information_L_Y, mutual_information_L_Y_error = L_Y_mi_estimator.predict(codes_pca_L, ds_train_Y.astype(np.int32),
                                                                                verbose=10)

In [ ]:
print("(X;L) mutual information: %.2f ± %.2f" % (mutual_information_X_L, mutual_information_X_L_error))
print("(L;Y) mutual information: %.2f ± %.2f" % (mutual_information_L_Y, mutual_information_L_Y_error))